In [ ]:
# DEPRECATED
# !gcloud auth login --enable-gdrive-access --update-adc


In [ ]:
!pip install pyspark
!pip install XlsxWriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 57.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=20c32c35983fece8b908e5ebce0ada39fac1baac25c4654976651f11d5dec5a5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.2 MB/s 


In [ ]:
# We must put into functions_v1.py
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from google.auth import default
from gspread.models import Worksheet ,Cell
import xlsxwriter
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import row_number, monotonically_increasing_id, desc, concat_ws
from pyspark.sql.functions import col, when, lit, regexp_replace, from_json, max, expr, first
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from typing import List
from google.oauth2.sts import Client
import pyspark.sql.functions as F
from xlsxwriter.utility import *
from datetime import date
from datetime import datetime
from datetime import timedelta 
import functools
from functools import reduce
from pyspark.sql.functions import when
from xlsxwriter.utility import xl_col_to_name
from google.auth.impersonated_credentials import Credentials

In [ ]:
!cp "/functions_v1.py" /content

In [ ]:
from functions_v1 import create_df, map_sheet, create_dataframe, filter_controm_df, filter_batch_df, join_df

In [ ]:
first_position: int = 0
today: date = date.today()
year: date = date.today().year

yesterday: date = today - timedelta(days = 1)
odate: str = yesterday.strftime('%Y%m%d')

#CHANGE EVERY MONTH
batch_report: str = '1hZARIq4JfaHqo70E3KS8RkMpD9jA0G1VHzQYPp8ugxw'
sheet_month: str = 'Julio 2022'
sheet_diary_work: str = 'Trabajo Diario-Julio'

In [ ]:
def create_gspread():
  auth.authenticate_user()
  gauth: GoogleAuth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive: GoogleDrive = GoogleDrive(gauth)
  creds, _ = default()
  return gspread.authorize(creds)

In [ ]:
def run_batch_dataFrame(batch_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_batch: Worksheet = gc.open_by_key(batch_id).worksheet(sheet_month)
  batch_df: DataFrame = create_dataframe(worksheet_batch,spark)
  return batch_df.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())))\
  .filter(batch_df["Desplanificado"]==False), worksheet_batch #.select("Job en Control-M","Job en Argos - Dataproc","index")
  

In [ ]:
def run_diarywork_dataFrame(diary_work_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_diary_work: Worksheet = gc.open_by_key(diary_work_id).worksheet(sheet_diary_work)
  diary_work_df: DataFrame = create_dataframe(worksheet_diary_work,spark)
  diary_work_df = diary_work_df.filter(~diary_work_df["Job en Control-M"].isin("Job en Control-M"))
  return diary_work_df, worksheet_diary_work

In [ ]:
def features_to_eav(batch_df:DataFrame, subset:str=None):

  id_vars = []

  spark: SparkSession = SparkSession.builder.getOrCreate()

  columns_batch: list[str] = batch_df.columns

  index_of_first_ODATE: int = columns_batch.index("Dependencia") + 1 

  number_of_odates_batch: int = len(columns_batch[index_of_first_ODATE:-1]) 

  odate_columns: list[str] = columns_batch[index_of_first_ODATE:] 

  for i in odate_columns:
    update_func: col = (F.when(F.col(i) == i, i+"a")
                .otherwise(F.col(i)))
    batch_df: DataFrame = batch_df.withColumn(i + "a", update_func)
    batch_df: DataFrame = batch_df.drop(i)

  columns_batch: list[str] = batch_df.columns 
  
  odate_columns: list[str] = columns_batch[index_of_first_ODATE:-1] 
  
  for i in columns_batch:
    if((i=="Job en Control-M")|(i=="Job en Argos - Dataproc")|(i=="Notas")|(i=="Acción")|(i=="Current_Date")|\
        (i=="Odate")|(i=="Estado")|(i=="Comentarios")):
      id_vars.append(i)     

  relevant_columns: str = subset if subset else batch_df.columns

  cols_to_stack: str = ", ".join(['\'{c}\', {c}'.format(c=c) for c in odate_columns]) 
  stack_expression: str = "stack({}, {}) as (ODATE, Estado)".format(number_of_odates_batch, cols_to_stack)

  features_to_check_df: None = batch_df.select(*(id_vars + odate_columns)).createOrReplaceTempView("features_to_check")

  
  sql1: str = "select `Job en Control-M`, `Job en Argos - Dataproc`, `Notas`, `Acción`, {} from features_to_check".format(stack_expression)
  print ("Stacking sql1:", sql1)

  return spark.sql(sql1)




In [ ]:
def filtered2(df1:DataFrame):
  
  unpivot_df_with_today: DataFrame = df1.withColumn("Current Date", lit(today).cast("string"))

  unpivot_df_with_today: DataFrame = unpivot_df_with_today.withColumn('ODATE', regexp_replace('ODATE', 'a', ''))

  # unpivot_df_with_today.show(5,False)

  return unpivot_df_with_today \
   .filter((unpivot_df_with_today["Estado"]=="REVISAR") | (unpivot_df_with_today["Estado"]=="RECLAMADO")|\
           (unpivot_df_with_today["Estado"]=="RELANZAR") | (unpivot_df_with_today["Estado"]=="RECLAMAR")|\
           (unpivot_df_with_today["Estado"]=="NO RAW") | (unpivot_df_with_today["Estado"]=="NO MASTER"))


In [ ]:
def filter_batch_dataFrame(df:DataFrame, first_position: int, diary_work_df: DataFrame):
  first_position = diary_work_df.count()

  df: DataFrame = df.withColumn('new_index', row_number().over(Window.orderBy(monotonically_increasing_id()))+first_position+2)
  
  return df


In [ ]:
def map_df_in_diary_worksheet(joined_index_df: DataFrame, worksheet_diary_batch: Worksheet, colu: str) -> None:
  try:    
    colum: str = xlsxwriter.utility.xl_col_to_name(worksheet_diary_batch.get_all_values()[0].index(colu))
    listIndex: List[tuple] = joined_index_df.select("new_index",str(colu)).where(col(colu).isNotNull()).rdd.map(tuple).collect()
    range: str = colum+"2:"+colum+"10000"
    cell_list: list[Cell] = worksheet_diary_batch.range(range)
    print(cell_list)
    i=0
    for cell in (cell for cell in cell_list if cell.row in list(map(lambda x:x[0],listIndex))):
      cell.value = listIndex[i][1]
      i=i+1
    worksheet_diary_batch.update_cells(cell_list)
  except:
    print(colum+" no esta en esta pagina")

In [ ]:
def map_diary_work_in_sheet(column_list: list, df:DataFrame, worksheet_to_map: Worksheet) -> None :
  for column_l in column_list:
    map_df_in_diary_worksheet(df,worksheet_to_map,column_l)

In [ ]:
def run_diary_work(batch_report:str) -> None :
  gc: Client = create_gspread()
  spark: SparkSession = SparkSession.builder.getOrCreate()

  batch_df , batch_sheet = run_batch_dataFrame(batch_report,spark,gc)
  diary_work_df, diary_work_sheet = run_diarywork_dataFrame(batch_report,spark,gc)

  columns_batch = batch_df.columns 
  index_of_first_ODATE = columns_batch.index("Dependencia") + 1
  odate_columns = columns_batch[index_of_first_ODATE:-1]
  unpivot_df: DataFrame = features_to_eav(batch_df, odate_columns)
  filtered_df: DataFrame = filtered2(unpivot_df)
  index_df: DataFrame = filter_batch_dataFrame(filtered_df, first_position, diary_work_df)
  index_df.show(20,False)
  worksheet_diary_work: Worksheet = gc.open_by_key(batch_report).worksheet(sheet_diary_work)
  columns_diary_work: list = diary_work_df.columns
  map_diary_work_in_sheet(columns_diary_work, index_df, worksheet_diary_work) 

In [ ]:
run_diary_work(batch_report)

Stacking sql1: select `Job en Control-M`, `Job en Argos - Dataproc`, `Notas`, `Acción`, stack(33, '20220629a', 20220629a, '20220630a', 20220630a, '20220701a', 20220701a, '20220702a', 20220702a, '20220703a', 20220703a, '20220704a', 20220704a, '20220705a', 20220705a, '20220706a', 20220706a, '20220707a', 20220707a, '20220708a', 20220708a, '20220709a', 20220709a, '20220710a', 20220710a, '20220711a', 20220711a, '20220712a', 20220712a, '20220713a', 20220713a, '20220714a', 20220714a, '20220715a', 20220715a, '20220716a', 20220716a, '20220717a', 20220717a, '20220718a', 20220718a, '20220719a', 20220719a, '20220720a', 20220720a, '20220721a', 20220721a, '20220722a', 20220722a, '20220723a', 20220723a, '20220724a', 20220724a, '20220725a', 20220725a, '20220726a', 20220726a, '20220727a', 20220727a, '20220728a', 20220728a, '20220729a', 20220729a, '20220730a', 20220730a, '20220731a', 20220731a) as (ODATE, Estado) from features_to_check
+---------------------------------------------------+---------------